In [ ]:
import great_expectations as gx
import pandas as pd

from great_expectations.data_context import FileDataContext

context = FileDataContext.create(project_root_dir=".")

# You can take a look at all configurations related to GX here
print(context)

In [ ]:
context.sources.add_pandas_filesystem(
    name="my_ds_2", base_directory="../data/yellow_tripdata/"
)

In [ ]:
my_ds = context.datasources["my_ds_2"]

my_batching_regex = "yellow_tripdata_2022-.*.parquet"

# Create the data asset (as one or more files from our data source)
my_asset = my_ds.add_parquet_asset(
    name="my_tripdata_data_asset", batching_regex=my_batching_regex
)

# Define a Batch Request to include all batches in the available data set
my_batch_request = my_asset.build_batch_request()
batches = my_asset.get_batch_list_from_batch_request(my_batch_request)

In [ ]:
for batch in batches:
    print(batch.batch_spec)

In [ ]:
asset_validator = context.get_validator(
    batch_request=my_batch_request,
    expectation_suite_name="default",
)
asset_validator.head()

In [ ]:
checkpoint = context.add_or_update_checkpoint(
    name="yellow_tripdata_asset_checkpoint",
    validator=asset_validator
)

# Get the result after validation
checkpoint_result = checkpoint.run()

# Quick view on the validation result
context.view_validation_result(checkpoint_result)